In [1]:
import argparse
import json
import os
import time
from copy import deepcopy
from datetime import datetime
from math import log, pi, sqrt
import sys

In [2]:
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchvision
import yaml
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch import nn
from torch.nn import functional as F
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from tqdm import tqdm

In [4]:
from data import generate_latent_grid, get_data
from laplace.onlinelaplace import OnlineLaplace
from laplace.posthoclaplace import PosthocLaplace
from models import get_decoder, get_encoder
from utils import compute_typicality_score, create_exp_name
from visualizer import (compute_and_plot_roc_curves, plot_calibration_plot,
                        plot_latent_space, plot_latent_space_ood,
                        plot_ood_distributions, plot_reconstructions,
                        save_metric)

ImportError: cannot import name '_grad_input_padding' from 'torch.nn.grad' (x:\miniforge\lib\site-packages\torch\nn\grad.py)

In [5]:
config = {
    # Experiment name
    'exp_name': "ood_experiment/",

    # Training settings
    'train': False,
    'posthoc': False,
    'alpha': 0,
    'train_samples': 1,
    'prior_precision': 1,

    # Test settings
    'ood': True,
    'test_samples': 100,

    # Data settings
    'dataset': "mnist",
    'ood_dataset': "kmnist",
    'batch_size': 50,

    # Model settings
    'pretrained': False,
    'no_conv': True,
    'latent_size': 2,

    # Hessian settings
    'backend': "layer",
    'approximation': "exact",  # options: block, exact, diag, mix
    'hessian_memory_factor': 0.999,
    'one_hessian_per_sampling': False,
    'update_hessian': True,
    'hessian_scale': 1
}


In [6]:
print(json.dumps(config, indent=4))
config["exp_name"] = create_exp_name(config)
print(config["exp_name"])

{
    "exp_name": "ood_experiment/",
    "train": false,
    "posthoc": false,
    "alpha": 0,
    "train_samples": 1,
    "prior_precision": 1,
    "ood": true,
    "test_samples": 100,
    "dataset": "mnist",
    "ood_dataset": "kmnist",
    "batch_size": 50,
    "pretrained": false,
    "no_conv": true,
    "latent_size": 2,
    "backend": "layer",
    "approximation": "exact",
    "hessian_memory_factor": 0.999,
    "one_hessian_per_sampling": false,
    "update_hessian": true,
    "hessian_scale": 1
}


NameError: name 'create_exp_name' is not defined

In [ ]:

# train or load auto encoder
if config["train"] and not config["posthoc"]:
    train_lae(config)

# fit laplace approximation post-hoc
elif config["train"] and config["posthoc"]:
    fit_lae(config)

test_lae(config)